In [2]:
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
tqdm.pandas()

In [4]:
data = pd.read_csv('../../Data/Projects/Qualifications_EN.csv')
data.head()

,Qualifications
0,"<p style=""text-align: center;"">\t<span style=""..."
1,"<p class=""MsoNormal"" text-indent:="""">\t<strong..."
2,"<span dir=""ltr""><span style=""FONT-SIZE: 10pt; ..."
3,"<p>\t<span style=""font-size: 11px;""><span styl..."
4,"<ul>\t<li>\t\t<span style=""font-size: 10pt""><s..."


In [13]:
def CleanHtml(text):
    if pd.isnull(text):
        return text
    return BeautifulSoup(text).getText(' ').strip() 

data['Qualifications_HtmlFree'] = data.progress_apply(lambda x: CleanHtml(x['Qualifications']),axis=1)

  7%|▋         | 1778/25000 [00:01<00:13, 1668.61it/s]/Users/kemalcankara/miniforge3/envs/nlpEnv/lib/python3.11/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
100%|██████████| 25000/25000 [00:15<00:00, 1602.55it/s]


In [14]:
jobTexts = data['Qualifications_HtmlFree'].tolist()
jobTexts[4]

'University degree in computer sciences or related fields      Fluency in\xa0 English      5\xa0years of experience in application development with Java ( C++ knowledge/experience preferred )      Knowledge of Object Oriented concepts and technologies, UML      Knowledge of software life cycle and methodologies      Experience on Multi-tier architectures, design patterns and Java BluePrints      Experience with J2EE technologies and web application development      Experience with one of the Object Relational Mapping (ORM) solutions      Spring framework experience is a plus     Adobe Flex knowdledge/experience is a plus      Strong analytical, interpersonal and problem solving skills      Strong technical documentation and verbal/written communication skills     Team player      No military service obligation for male candidates \t\xa0 İŞ TANIMI The candidate will be a member of a team that is responsible for;     Analyzing requirements     Building use cases      Architecting and des

In [15]:
with open('../../Data/jobTexts.txt', 'w') as f:
    for line in jobTexts:
        f.write(f"{line}\n\n")

In [16]:
jobTexts[10]

'Triodor is a Holland based international Software R&D company that produces projects for our customers from all over Europe, such as Holland, Germany and Switzerland. We\xa0 are looking for  Senior Software Developers \xa0 who can adopt to our young & energetic\xa0 team and dynamic working atmosphere and also\xa0 have no restriction\xa0 to travel abroad. The Role Developing projects with ASP .Net/C# languages on MSSQL Server, involving to analyses,\xa0 preparing functional documents, participating in on-site meetings. \xa0 Main Responsibilities    Develops and maintains in-depth understanding of client’s existing business processes and applications   Reviews and contributes to the definition of functional requirements   Identifies technical options for providing operational/?production support or resolving applications or system problems   Reviews system performance and recommends approaches improving overall performance   Participate in detailed project planning and design reviews.? 

In [3]:
from transformers import GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, GPT2LMHeadModel, pipeline, \
                         Trainer, TrainingArguments

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')  # load up a standard gpt2 model

tokenizer.pad_token = tokenizer.eos_token  
# set our pad token to be the eos token. This lets gpt know how to fill space

In [5]:
# load up our data into a dataset
pds_data = TextDataset(
    tokenizer=tokenizer,
    file_path='../../Data/jobTexts.txt',  # IT JOBS 25K English
    block_size=64  # length of each chunk of text to use as a datapoint
)

/Users/kemalcankara/miniforge3/envs/nlpEnv/lib/python3.11/site-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [6]:
pds_data[0], pds_data[0].shape  # inspect the first point

(tensor([30464,  1236, 25299,   318,   530,   286,   262,   995,   338,  3756,
         11521,  4508,  8062,  5745,   532,   287,  3298,  2546,    11,   287,
          4708,  3081,   290,   287,   262,  1271,   286,  7534,   356,  4691,
           351,  3294,  8062,  2594,    13,  6288,    11,   356,  8076,   517,
           621, 11323,  2678,   290,   423,  3298,  7686,  1973,   257,  1336,
          2837,  7124,  8062, 16020,    11, 13630,  4050,  6946,   523,   297,
          3508,   329,  1957,   290]),
 torch.Size([64]))

In [7]:
print(tokenizer.decode(pds_data[0]))

McCann Istanbul is one of the world's leading integrated brand communications organizations - in global size, in professional quality and in the number of clients we serve with multiple communications services. Today, we operate more than 130 countries and have global networks across a full range marketing communications sectors, delivering effective communication sollutions for local and


In [8]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,  
    # MLM is Masked Language Modelling (for BERT + auto-encoding tasks)
)

In [9]:
model = GPT2LMHeadModel.from_pretrained('gpt2')  # load up a GPT2 model

pretrained_generator = pipeline(  # create a generator with built in params
    'text-generation', model=model, tokenizer='gpt2',
    config={'max_length': 200, 'do_sample': True, 'top_p': 0.9, 'temperature': 0.7, 'top_k': 10}
)

In [10]:
print('----------')
for generated_sequence in pretrained_generator('Candidates must have graduated', num_return_sequences=3):
    print(generated_sequence['generated_text'])
    print('----------')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


----------


/Users/kemalcankara/miniforge3/envs/nlpEnv/lib/python3.11/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 50 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Candidates must have graduated from high school, the same way they received federal Pell Grant scholarships.

"It's good policy because it enables a school to attract, recruit and retain the highest quality of students and to help these students graduate through a
----------
Candidates must have graduated from a public-private college before they became an official, and that is the exception to the Constitution's guarantee of constitutional free elections.

As a first term senator since 1990, Paul has voted as chairman of the Senate
----------
Candidates must have graduated from college, and have already completed at least 20 years in an official position. Under certain circumstances, such as when a candidate is a new recruit or a candidate for a promotion to a position of higher ranking or higher need for
----------


In [11]:
print('----------')
for generated_sequence in pretrained_generator('Experienced in machine learning skills like ', num_return_sequences=3):
    print(generated_sequence['generated_text'])
    print('----------')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


----------
Experienced in machine learning skills like ike, we find that our findings are significant and significant not due to any natural differences in training level. While more research is needed to make sure that machines learn efficiently and accurately over time, we can say with
----------
Experienced in machine learning skills like 𝐘 (where 𝐨 is the most general term, of course) and  (where 𝐐⟥ is the most general term). 𝓉 uses a simple
----------
Experienced in machine learning skills like 〜A (R(x))〜, (R(x))〜 for all x. For simplicity I will assume each t-values are 1∶x

Let R R and
----------


In [12]:
print('----------')
for generated_sequence in pretrained_generator('Experienced in machine learning skills like', num_return_sequences=3):
    print(generated_sequence['generated_text'])
    print('----------')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


----------
Experienced in machine learning skills like machine learning on iOS, Android, and iPhone or using OpenOffice, open source, Django, NodeJS, Django 2.5, or many other open source projects, and using a Python script for the python tool
----------
Experienced in machine learning skills like machine learning with algorithms from academia and industry, Dr. Jens Hoeniger is a Senior Professor in Psychology at the University of South Carolina. His graduate work includes the investigation of the relationship between brain function and cognitive
----------
Experienced in machine learning skills like Bayesian inference and Bayesian inference algorithms, I feel I'm the best fit for an initial job.

I will have a few rounds of interviews at the end of the summer before starting to test work.
----------


In [13]:
import os 
os.environ["WANDB_DISABLED"]="true"

In [14]:
training_args = TrainingArguments(
    output_dir="../../Data/Weights/gpt2_pds", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=32,  # batch size for evaluation
    logging_steps=10,
    load_best_model_at_end=True,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    report_to='none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=pds_data.examples[:int(len(pds_data.examples)*.8)],
    eval_dataset=pds_data.examples[int(len(pds_data.examples)*.8):]
)

trainer.evaluate()

***** Running Evaluation *****
  Num examples = 30206
  Batch size = 32


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/944 [00:00<?, ?it/s]

{'eval_loss': 4.483351707458496,
 'eval_runtime': 2123.0298,
 'eval_samples_per_second': 14.228,
 'eval_steps_per_second': 0.445}

In [15]:
trainer.train()

/Users/kemalcankara/miniforge3/envs/nlpEnv/lib/python3.11/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 120821
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 11328
  Number of trainable parameters = 124439808


  0%|          | 0/11328 [00:00<?, ?it/s]

{'loss': 4.3882, 'learning_rate': 4.9955861581920905e-05, 'epoch': 0.0}
{'loss': 4.155, 'learning_rate': 4.991172316384181e-05, 'epoch': 0.01}
{'loss': 3.9985, 'learning_rate': 4.986758474576271e-05, 'epoch': 0.01}
{'loss': 3.9261, 'learning_rate': 4.982344632768362e-05, 'epoch': 0.01}
{'loss': 3.8922, 'learning_rate': 4.9779307909604523e-05, 'epoch': 0.01}
{'loss': 3.8202, 'learning_rate': 4.9735169491525426e-05, 'epoch': 0.02}
{'loss': 3.809, 'learning_rate': 4.969103107344633e-05, 'epoch': 0.02}
{'loss': 3.8269, 'learning_rate': 4.964689265536723e-05, 'epoch': 0.02}
{'loss': 3.6598, 'learning_rate': 4.960275423728814e-05, 'epoch': 0.02}
{'loss': 3.7811, 'learning_rate': 4.9558615819209044e-05, 'epoch': 0.03}
{'loss': 3.7218, 'learning_rate': 4.951447740112995e-05, 'epoch': 0.03}
{'loss': 3.7033, 'learning_rate': 4.947033898305085e-05, 'epoch': 0.03}
{'loss': 3.7716, 'learning_rate': 4.942620056497175e-05, 'epoch': 0.03}
{'loss': 3.7531, 'learning_rate': 4.938206214689266e-05, 'epoch

KeyboardInterrupt: 

In [ ]:
trainer.evaluate()  

In [ ]:
trainer.save_model()

In [ ]:
loaded_model = GPT2LMHeadModel.from_pretrained('./gpt2_pds')

finetuned_generator = pipeline(
    'text-generation', model=loaded_model, tokenizer=tokenizer,
    config={'max_length': 200, 'do_sample': True, 'top_p': 0.9, 'temperature': 0.7, 'top_k': 10}
)

In [ ]:
print('----------')
for generated_sequence in pretrained_generator('Experienced in machine learning skills like', num_return_sequences=3):
    print(generated_sequence['generated_text'])
    print('----------')

https://huggingface.co/docs/transformers/model_doc/gptj

https://huggingface.co/bigscience/bloom?text=Experienced+in+machine+learning+skills+like+deep+learning%2C+neural+networks%2C+and+reinforcement+learning.%0AExperienced+in+designing+and+implementing+machine+learning